In [1]:
import ccxt
import talib

from datetime import date, datetime
import time

import pandas as pd
import numpy as np

import plotly
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import plotly.graph_objs as go
init_notebook_mode(connected=True)

pd.set_option('display.max_rows', None)

# Candlestick Patterns Detection


* [1. Obtaining Historical Data](#1-bullet)
* [2. Patterns Recognition](#2-bullet)
* [3. Dive Into BTC ](#3-bullet)
* [4. Altcoins Summary](#4-bullet)
* [5. Results](#5-bullet)

## Obtaining historical data <a id='1-bullet'></a>

Getting prices from January 2021 as this is the starting point of the current cycle

In [2]:
coins = ['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'DOGEUSDT', 'NEARUSDT',
         'ONEUSDT', 'BNBUSDT', 'XRPUSDT', 'ADAUSDT', 'DOTUSDT',
         'ATOMUSDT', 'FTTUSDT', 'LTCUSDT', 'AVAXUSDT', 'MATICUSDT']

In [3]:
binance = ccxt.binance()

startDate = "2021-01-01"
startDate = datetime.strptime(startDate, "%Y-%m-%d")
startDate = datetime.timestamp(startDate)
startDate = int(startDate) * 1000

if binance.has['fetchOHLCV']:
    for symbol in coins:
        ohlcv = binance.fetch_ohlcv(symbol, '4h', since=startDate, limit=1000)
        #ohlcv = ohlcv[:-1] # removing last - still open - candlestick
        vars()[symbol] = pd.DataFrame(ohlcv, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
        vars()[symbol]['Time'] = [datetime.fromtimestamp(float(time)/1000) for time in vars()[symbol]['Time']]

In [4]:
BTCUSDT.head()

,Time,Open,High,Low,Close,Volume
0,2021-01-01 03:00:00,28923.63,29470.00,28690.17,29278.40,11560.456553
1,2021-01-01 07:00:00,29278.41,29395.00,28806.54,29092.83,7308.910274
2,2021-01-01 11:00:00,29092.84,29402.57,28872.24,29313.49,8283.705319
3,2021-01-01 15:00:00,29313.49,29600.00,29030.14,29188.67,11794.949515
4,2021-01-01 19:00:00,29188.67,29360.00,28624.57,29029.04,9850.965345


In [5]:
#BTCUSDT.to_csv('BTCUSDT_4h.csv', index=False)

## Patterns Recognition <a id='2-bullet'></a>
 
Now let's detect patterns

[TA-lib](https://mrjbq7.github.io/ta-lib/func_groups/pattern_recognition.html)

In [6]:
crypto_df = pd.read_csv('BTCUSDT_4h.csv')
crypto_df.head()

,Time,Open,High,Low,Close,Volume
0,2021-01-01 03:00:00,28923.63,29470.00,28690.17,29278.40,11560.456553
1,2021-01-01 07:00:00,29278.41,29395.00,28806.54,29092.83,7308.910274
2,2021-01-01 11:00:00,29092.84,29402.57,28872.24,29313.49,8283.705319
3,2021-01-01 15:00:00,29313.49,29600.00,29030.14,29188.67,11794.949515
4,2021-01-01 19:00:00,29188.67,29360.00,28624.57,29029.04,9850.965345


In [7]:
# list of all available patterns
pattern_names = talib.get_function_groups()['Pattern Recognition']

In [8]:
# Extract OHLC data
open = crypto_df['Open']
high = crypto_df['High']
low = crypto_df['Low']
close = crypto_df['Close']

In [9]:
def seeking_patterns(dataframe):
    
    for pattern in pattern_names:
        dataframe[pattern] = getattr(talib, pattern)(open, high, low, close)
        
    for i in dataframe.index:
        for cd in pattern_names:
            if dataframe.loc[i, cd] == -100:
                dataframe.loc[i, cd] = 'Bearish'
            if dataframe.loc[i, cd] == 100:
                dataframe.loc[i, cd] = 'Bullish'
                
    return dataframe

In [10]:
pd.set_option('display.max_columns', None)
crypto_df_patterns = seeking_patterns(crypto_df)
crypto_df_patterns.head()

,Time,Open,High,Low,Close,Volume,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLGAPSIDESIDEWHITE,CDLGRAVESTONEDOJI,CDLHAMMER,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLHIGHWAVE,CDLHIKKAKE,CDLHIKKAKEMOD,CDLHOMINGPIGEON,CDLIDENTICAL3CROWS,CDLINNECK,CDLINVERTEDHAMMER,CDLKICKING,CDLKICKINGBYLENGTH,CDLLADDERBOTTOM,CDLLONGLEGGEDDOJI,CDLLONGLINE,CDLMARUBOZU,CDLMATCHINGLOW,CDLMATHOLD,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLONNECK,CDLPIERCING,CDLRICKSHAWMAN,CDLRISEFALL3METHODS,CDLSEPARATINGLINES,CDLSHOOTINGSTAR,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS
0,2021-01-01 03:00:00,28923.63,29470.00,28690.17,29278.40,11560.456553,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2021-01-01 07:00:00,29278.41,29395.00,28806.54,29092.83,7308.910274,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2021-01-01 11:00:00,29092.84,29402.57,28872.24,29313.49,8283.705319,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2021-01-01 15:00:00,29313.49,29600.00,29030.14,29188.67,11794.949515,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2021-01-01 19:00:00,29188.67,29360.00,28624.57,29029.04,9850.965345,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
#crypto_df_patterns.to_csv('btc_patterns_4h.csv', index=False)

Now is the time to see how the patterns helped to predict the price movement. I am looking for strong reversal signals, so it is not enough for the pattern to show a good result only on the next candle. The following 4 candles (16 hours) will be taken into account.

**candle_test()** function will make calculations easier
- if *bullish* pattern: result = next selected timeframe close - signal candlestick close
- if *bearish*: result = signal candlestick close - next selected timeframe close

In [12]:
candle_patterns_stat = []

def candle_test(df, pattern, period):
    
    """dataframe with patterns
    pattern to create new df with it
    look-forward period in candles"""
    
    profit = 0
    losses = 0
    trade_results_all = []
    trade_results_all_prc = []
    minus_trades = []
    plus_trades = []
    
    new_df = df[['Time','Close', pattern]].copy()
    
    for i, row in new_df.iterrows():
        if row[pattern] == 'Bullish':
            try:
                index_number = i
                difference = new_df['Close'].iloc[i + period] - new_df['Close'].iloc[i]
                #percent_diff = ((new_df['Close'].iloc[i + 1] - new_df['Close'].iloc[i])/new_df['Close'].iloc[i]) * 100
                if difference > 0:
                    profit += 1
                else:
                    losses += 1
                trade_results_all.append(difference)
                #trade_results_prc.append(percent_diff)
                if difference > 0:
                    plus_trades.append(difference)
                else:
                    minus_trades.append(difference)
            except IndexError:
                pass
            continue
            
        if row[pattern] == 'Bearish':
            try:
                index_number = i
                difference = new_df['Close'].iloc[i] - new_df['Close'].iloc[i + period]
                #percent_diff = ((new_df['Close'].iloc[i] - new_df['Close'].iloc[i + 1])/new_df['Close'].iloc[i]) * 100
                if difference > 0:
                    profit += 1
                else:
                    losses += 1
                trade_results_all.append(difference)
                #trade_results_prc.append(percent_diff)
                if difference > 0:
                    plus_trades.append(difference)
                else:
                    minus_trades.append(difference)
            except IndexError:
                pass
            continue
            
        sorted_plus_trades = sorted(plus_trades)
        sorted_plus_trades = [round(num, 2) for num in sorted_plus_trades]

        sorted_minus_trades = sorted(minus_trades, reverse=True)
        sorted_minus_trades = [round(num, 2) for num in sorted_minus_trades]

    num_trades = (profit + losses)

    candle_patterns_stat.append({
        'name': pattern,
        'num_trades': num_trades,
        'total_result': round(sum(trade_results_all), 2),
        'profit_trades': profit,
        'loss_trades': losses
    })    
    
    return candle_patterns_stat

In [13]:
for pattern in pattern_names:
    df = pd.DataFrame(candle_test(crypto_df_patterns, pattern, 1))

In [14]:
# selecting only excisting patterns
worked_patterns = df.loc[df['num_trades'] > 0].copy()
print(f"Patterns in work: {worked_patterns.shape[0]}")

Patterns in work: 38


In [15]:
worked_patterns['average_profit'] = round(worked_patterns['total_result'] / worked_patterns['profit_trades'])
worked_patterns['prc_good_trades'] = round((worked_patterns['profit_trades'] / worked_patterns['num_trades']) * 100)


worked_patterns.sort_values(['prc_good_trades'], ascending=False, inplace=True)
worked_patterns.head()

,name,num_trades,total_result,profit_trades,loss_trades,average_profit,prc_good_trades
19,CDLEVENINGDOJISTAR,1,1624.13,1,0,1624.0,100.0
47,CDLRISEFALL3METHODS,1,406.52,1,0,407.0,100.0
3,CDL3LINESTRIKE,10,3885.22,8,2,486.0,80.0
60,CDLXSIDEGAP3METHODS,20,5587.38,14,6,399.0,70.0
29,CDLHIKKAKEMOD,3,243.55,2,1,122.0,67.0


In [16]:
#worked_patterns.to_csv('btc_4h_good_on_1tf.csv', index=False)

## Dive Into BTC <a id='3-bullet'></a>


In [17]:
def new_df(dataframe, pattern):
    
    """
    generating new dataframe for specified pattern
    
    """
    
    new_df = dataframe[['Time', 'Open', 'High', 'Low', 'Close', pattern]].copy()
    new_df['year_week'] = pd.to_datetime(new_df['Time']).dt.strftime('%Y-%U')
    
    return new_df

In [18]:
# uploading all 4 dataframes with calculated results 

df1tf = pd.read_csv('btc_4h_good_on_1tf.csv')
df2tf = pd.read_csv('btc_4h_good_on_2tf.csv')
df3tf = pd.read_csv('btc_4h_good_on_3tf.csv')
df4tf = pd.read_csv('btc_4h_good_on_4tf.csv')

price = pd.read_csv('btc_patterns_4h.csv')

Despite 35 spotted patterns only 11 of them have more than 50% of good trades. That's not that bad actually. Traders deal with a high level of uncertainty and even a 50% probability allows to make money in the market if the rules of risk management are followed. Besides, we need signals for informative purposes, not for algorithmic trading, so the main goal is to get less bad signals. 

In [19]:
df1tf.loc[df1tf['prc_good_trades'] > 50]

,name,num_trades,total_result,profit_trades,loss_trades,average_profit,prc_good_trades
0,CDLEVENINGDOJISTAR,1,1624.13,1,0,1624.0,100.0
1,CDLRISEFALL3METHODS,1,406.52,1,0,407.0,100.0
2,CDL3LINESTRIKE,10,3885.22,8,2,486.0,80.0
3,CDLXSIDEGAP3METHODS,20,5587.38,14,6,399.0,70.0
4,CDLHIKKAKEMOD,3,243.55,2,1,122.0,67.0
5,CDLSHOOTINGSTAR,13,824.41,8,5,103.0,62.0
6,CDLHIKKAKE,434,-6479.55,232,202,-28.0,53.0
7,CDLHARAMI,137,2159.56,71,66,30.0,52.0
8,CDLSEPARATINGLINES,23,3042.46,12,11,254.0,52.0
9,CDLHIGHWAVE,538,-8956.18,278,260,-32.0,52.0


In [20]:
print(f"More than 50% of good signals: {df1tf.loc[df1tf['prc_good_trades'] > 50]['name'].count()}")
print(f"Less than 50% of good signals: {df1tf.loc[df1tf['prc_good_trades'] <= 50]['name'].count()}")

More than 50% of good signals: 11
Less than 50% of good signals: 27


In [21]:
# aggrigation of all results
pdList = [df1tf, df2tf, df3tf, df4tf]

btc_agg = (
    pd.concat(pdList)
    .groupby('name', as_index=False).agg({'prc_good_trades': 'mean', 'num_trades': 'median'})
    .sort_values('prc_good_trades', ascending=False)
)


btc50 = btc_agg.loc[btc_agg['prc_good_trades'] > 50]
btc50

,name,prc_good_trades,num_trades
10,CDLEVENINGDOJISTAR,100.00,1.0
1,CDL3LINESTRIKE,65.00,10.0
37,CDLXSIDEGAP3METHODS,57.50,20.0
32,CDLSHOOTINGSTAR,56.00,13.0
15,CDLHANGINGMAN,56.00,29.0
19,CDLHIKKAKE,54.00,434.0
7,CDLDOJISTAR,52.50,30.0
26,CDLMATCHINGLOW,51.25,65.0
13,CDLGRAVESTONEDOJI,51.25,47.0
18,CDLHIGHWAVE,51.00,537.5


In [22]:
set(df1tf.loc[df1tf['prc_good_trades'] > 50]['name']) & set(btc50['name'])

{'CDL3LINESTRIKE',
 'CDLEVENINGDOJISTAR',
 'CDLHARAMI',
 'CDLHIGHWAVE',
 'CDLHIKKAKE',
 'CDLSHOOTINGSTAR',
 'CDLXSIDEGAP3METHODS'}

Only 7 patterns remained from df1tf (best patterns selected on next 4h timeframe). Now we have less chance that some patterns worked due to a random short term movement

In [23]:
print(f"More than 50% of good signals: {btc_agg.loc[btc_agg['prc_good_trades'] > 50]['name'].count()}")
print(f"Less than 50% of good signals: {btc_agg.loc[btc_agg['prc_good_trades'] <= 50]['name'].count()}")

More than 50% of good signals: 11
Less than 50% of good signals: 27


This is good because some of the patterns appear on the chart very often, but give false signals in more than half of the cases

In [24]:
btc_agg.loc[btc_agg['prc_good_trades'] <= 50]

,name,prc_good_trades,num_trades
22,CDLINVERTEDHAMMER,50.00,8.0
30,CDLRISEFALL3METHODS,50.00,1.0
20,CDLHIKKAKEMOD,50.00,3.0
8,CDLDRAGONFLYDOJI,49.50,51.0
36,CDLTAKURI,49.50,50.0
17,CDLHARAMICROSS,49.50,39.0
23,CDLLONGLEGGEDDOJI,49.25,574.5
6,CDLDOJI,49.25,576.5
4,CDLBELTHOLD,49.25,385.0
3,CDLADVANCEBLOCK,49.00,40.0


Let's take a look at some patterns with good profit trades to all trades ratio

### Evening doji star

Bearish reversal pattern:

- First candle in an uptrend (green body)
- Second candle - a doji candle above the previous candle body and low price below the previous candle high price
- Third candle (red body) below the previous candle body and closing price below the midpoint of the first candle body 

Very rare - only 1 appearance in two and a half years

In [25]:
evening_doji_star = new_df(price, 'CDLEVENINGDOJISTAR')

In [26]:
evening_doji_star.loc[(evening_doji_star['CDLEVENINGDOJISTAR'] == 'Bullish') \
                                      | (evening_doji_star['CDLEVENINGDOJISTAR'] == 'Bearish')]

,Time,Open,High,Low,Close,CDLEVENINGDOJISTAR,year_week
1026,2021-06-21 03:00:00,35600.17,35750.0,34566.82,34649.63,Bearish,2021-25


In [27]:
# marking signals

def bear_evening_doji_star(row):
    
    if row['CDLEVENINGDOJISTAR'] == 'Bearish':
        return row['Open'] * 1.02
    else:
        return np.nan  
    
evening_doji_star['evening_doji_star_bearish_dot'] = evening_doji_star.apply(bear_evening_doji_star, axis=1)

In [28]:
evening_doji_plot = evening_doji_star.loc[evening_doji_star['year_week'] == '2021-25'].copy()

In [29]:
fig = go.Figure()

fig.add_trace(go.Candlestick(x=evening_doji_plot['Time'],
                             open=evening_doji_plot['Open'],
                             high=evening_doji_plot['High'],
                             low=evening_doji_plot['Low'],
                             close=evening_doji_plot['Close'], opacity=0.5, name="Price"))


fig.add_trace(go.Scatter(x=evening_doji_plot['Time'], 
                         y=evening_doji_plot['evening_doji_star_bearish_dot'],
                         opacity=1, mode='markers', marker=dict(color='#16FF32', symbol='6', size=7, line=dict(color='MediumPurple',width=2)),
                         name="Bearish"))


fig.update_layout(xaxis_rangeslider_visible=False, title = 'Evening Doji Star')
iplot(fig, show_link=False)

![pattern](https://lh3.googleusercontent.com/ioKkt5gJFHAKKkJV5uRHnI5HMCkRsPmszjq3ozxsWEXVZKzkrWvQWxiCBPO3NwH6pEY1HTQMFKGO2dbPuGJvAnQG00AAdtWuCdqWcD0iaukYVlcT7D8m2xihcdpIKEQH4zCaSr2DezzzX_H5CYdLeWo)

### Three-Line Strike

Trend continuation pattern 

The bearish Three Line Strike:

- The first three candles are long and bearish and continue the downtrend having Close prices consequently lower
- The second and the third candles open within the previous candle's body
- The fourth candle is bullish, opening at a new Low price
- The fourth candle's Close price is higher than the Open price of the first candle

The bullish Three Line Strike:

- The first three candles are long and bullish and continue the uptrend having Close prices consequently higher
- The second and the third candles open within the previous candle's body
- The fourth candle is bearish, opening at a new High price
- The fourth candle's Close price is lower than the Open price of the first candle


In [30]:
three_line_strike = new_df(price, 'CDL3LINESTRIKE')

In [31]:
three_line_strike.loc[(three_line_strike['CDL3LINESTRIKE'] == 'Bullish') \
                                      | (three_line_strike['CDL3LINESTRIKE'] == 'Bearish')]

,Time,Open,High,Low,Close,CDL3LINESTRIKE,year_week
459,2021-03-18 15:00:00,58168.69,59637.39,57500.00,59533.68,Bearish,2021-11
473,2021-03-20 23:00:00,59272.83,59381.70,57966.85,58102.28,Bullish,2021-11
1067,2021-06-27 23:00:00,32815.18,34749.00,32357.13,34700.34,Bearish,2021-26
1767,2021-10-22 15:00:00,63376.81,63709.50,60478.10,60690.94,Bullish,2021-42
2092,2021-12-15 19:00:00,46632.30,49259.98,46547.00,48674.90,Bearish,2021-50
2254,2022-01-11 19:00:00,41733.92,43100.00,41680.44,42972.05,Bearish,2022-02
2584,2022-03-07 19:00:00,39071.49,39547.57,37192.78,37308.94,Bullish,2022-10
2616,2022-03-13 03:00:00,38807.35,39276.28,38703.73,39194.08,Bearish,2022-11
2774,2022-04-08 11:00:00,43763.53,43970.62,43260.76,43298.79,Bullish,2022-14
3193,2022-06-17 07:00:00,20383.37,21286.01,20326.00,21100.64,Bearish,2022-24


In [32]:
def bear_three_line_strike(row):
    
    if row['CDL3LINESTRIKE'] == 'Bearish':
        return row['Close'] * 1.02
    else:
        return np.nan  
    
def bull_three_line_strike(row):
    
    if row['CDL3LINESTRIKE'] == 'Bullish':
        return row['Open'] * 0.98
    else:
        return np.nan
    
three_line_strike['three_line_strike_bearish_dot'] = three_line_strike.apply(bear_three_line_strike, axis=1)    
three_line_strike['three_line_strike_bullish_dot'] = three_line_strike.apply(bull_three_line_strike, axis=1)

In [33]:
three_line_strike_plot_bear = three_line_strike.loc[three_line_strike['year_week'] == '2022-24'].copy()

In [34]:
fig = go.Figure()

fig.add_trace(go.Candlestick(x=three_line_strike_plot_bear['Time'],
                             open=three_line_strike_plot_bear['Open'],
                             high=three_line_strike_plot_bear['High'],
                             low=three_line_strike_plot_bear['Low'],
                             close=three_line_strike_plot_bear['Close'], name="Price", opacity=0.5))


fig.add_trace(go.Scatter(x=three_line_strike_plot_bear['Time'], 
                         y=three_line_strike_plot_bear['three_line_strike_bearish_dot'],
                         opacity=1, mode='markers', marker=dict(color='#16FF32', symbol='6', size=7, line=dict(color='MediumPurple',width=2)),
                         name="Bearish"))


fig.update_layout(xaxis_rangeslider_visible=False, title = 'Three-Line Strike bearish')
fig.show()

![pattern](https://lh3.googleusercontent.com/iaoN5ErdwUgpx7-jTVaG70J1sNHpmxSw-9CVSyp3EtJ9FFQ2PxUe5tloBC3WRWXsccbQnCOOwXuQR2B7T1l7XMvgP4TR9EFBq_i54o17ngvabChxbJrBNjVtVd8gswCDV2J8X6ChyteTk2bf2dtaxAY)

In [35]:
three_line_strike_plot_bull = three_line_strike.loc[three_line_strike['year_week'] == '2022-14'].copy()

In [36]:
fig = go.Figure()

fig.add_trace(go.Candlestick(x=three_line_strike_plot_bull['Time'],
                             open=three_line_strike_plot_bull['Open'],
                             high=three_line_strike_plot_bull['High'],
                             low=three_line_strike_plot_bull['Low'],
                             close=three_line_strike_plot_bull['Close'], name="Price", opacity=0.5))


fig.add_trace(go.Scatter(x=three_line_strike_plot_bull['Time'], 
                         y=three_line_strike_plot_bull['three_line_strike_bullish_dot'],
                         opacity=1, mode='markers', marker=dict(color='#16FF32', symbol='5', size=7, line=dict(color='MediumPurple',width=2)),
                         name="Bullish"))


fig.update_layout(xaxis_rangeslider_visible=False, title = 'Three-Line Strike bullish')
iplot(fig, show_link=False)

![pattern](https://lh6.googleusercontent.com/rI1FL1pdVIYNCuUmaphr22u13ZAt_U-JoQt-ObmbdIEMHjHJZnMNf8-iv1sBBBAqil2PODU3e2d-RLL0zWPlbmFB5fluV5aelr8z1v6ef0PFwOxExElT6ndmZinu4vxJ_wc4P9NgM5gaBNErAYAGrDQ)

### Hikkake Pattern

Is used to identify a short-term trend

The bearish Three Line Strike:

- The first bar completely overshadows the second bar
- The third bar includes a higher high and higher low than the second bar
- The fourth bar, or the fourth and fifth bars, drifts above the third bar
- The fifth or sixth bar’s body closes below the second bar’s low

The bullish Three Line Strike:

- The first bar completely overshadows the body of the second bar
- The third bar includes a lower high and lower low than the second bar
- The fourth bar, or the fourth and fifth bars, drifts below the third bar
- The fifth or sixth bar’s body closes above the second bar’s high

In [37]:
hikkake_pattern = new_df(price, 'CDLHIKKAKE')

In [38]:
hikkake_pattern.loc[(hikkake_pattern['CDLHIKKAKE'] == 'Bullish') \
                                      | (hikkake_pattern['CDLHIKKAKE'] == 'Bearish')].tail(10)

,Time,Open,High,Low,Close,CDLHIKKAKE,year_week
3299,2022-07-04 23:00:00,19829.98,20354.01,19733.95,20236.71,Bearish,2022-27
3302,2022-07-05 11:00:00,20215.68,20215.69,19478.00,19520.39,Bullish,2022-27
3313,2022-07-07 07:00:00,20480.60,20493.72,20251.68,20440.01,Bullish,2022-27
3320,2022-07-08 11:00:00,21797.65,21822.10,21317.00,21583.98,Bullish,2022-27
3323,2022-07-08 23:00:00,21811.90,22045.91,21580.71,21594.75,Bearish,2022-27
3328,2022-07-09 19:00:00,21545.89,21980.00,21517.87,21621.40,Bearish,2022-27
3330,2022-07-10 03:00:00,21592.15,21607.65,21091.81,21294.98,Bullish,2022-28
3332,2022-07-10 11:00:00,21339.46,21424.67,21161.67,21301.84,Bearish,2022-28
3341,2022-07-11 23:00:00,20509.47,20565.05,19875.23,19963.61,Bullish,2022-28
3353,2022-07-13 23:00:00,19652.15,20366.61,19584.37,20234.87,Bearish,2022-28


In [39]:
def bear_hikkake_pattern(row):
    
    if row['CDLHIKKAKE'] == 'Bearish':
        return (row['Open'] + row['Close']) / 2
    else:
        return np.nan  
    
def bull_hikkake_pattern(row):
    
    if row['CDLHIKKAKE'] == 'Bullish':
        return (row['Open'] + row['Close']) / 2
    else:
        return np.nan
    
hikkake_pattern['hikkake_pattern_bearish_dot'] = hikkake_pattern.apply(bear_hikkake_pattern, axis=1)    
hikkake_pattern['hikkake_pattern_bullish_dot'] = hikkake_pattern.apply(bull_hikkake_pattern, axis=1)

In [40]:
hikkake_pattern_plot = hikkake_pattern.loc[hikkake_pattern['year_week'] == '2022-28'].copy()

In [41]:
fig = go.Figure()

fig.add_trace(go.Candlestick(x=hikkake_pattern_plot['Time'],
                             open=hikkake_pattern_plot['Open'],
                             high=hikkake_pattern_plot['High'],
                             low=hikkake_pattern_plot['Low'],
                             close=hikkake_pattern_plot['Close'], name="Price", opacity=0.5))


fig.add_trace(go.Scatter(x=hikkake_pattern_plot['Time'], 
                         y=hikkake_pattern_plot['hikkake_pattern_bearish_dot'],
                         opacity=1, mode='markers', marker=dict(color='#16FF32', symbol='6', size=7, line=dict(color='MediumPurple',width=2)),
                         name="Bearish"))

fig.add_trace(go.Scatter(x=hikkake_pattern_plot['Time'], 
                         y=hikkake_pattern_plot['hikkake_pattern_bullish_dot'],
                         opacity=1, mode='markers', marker=dict(color='#16FF32', symbol='5', size=7, line=dict(color='MediumPurple',width=2)),
                         name="Bullish"))

fig.update_layout(xaxis_rangeslider_visible=False, title = 'Hikkake Pattern')
iplot(fig, show_link=False)

![pattern](https://lh4.googleusercontent.com/8iJ4BLEW0TiGaTcJOw5CBnHoUbIj4Q7rFBjlDX7Db_c3ytDK27GhZYZcvhqWSCaALRhzWirxZVuA670FnzyTfcYJd8p_FKVI2FiPN3l20BNndt4HZnrfz3uIMJaZFwzMiwhmG5taLWOk8Yi6gZrD4Rs)

Bullish pattern worked in 52.53% and bearish - in 54.24%. The main problem with hikkake pattern is that it is trying to detect a short-term trend in the opposite direction from the global trend.

In [42]:
plus_bull_trades = 0
trade_count = 0
for i, row in hikkake_pattern.iterrows():
    if row['CDLHIKKAKE'] == 'Bullish':
        difference = hikkake_pattern['Close'].iloc[i + 1] - hikkake_pattern['Close'].iloc[i]
        trade_count += 1
        if difference > 0:
            plus_bull_trades += 1

print(f'All bullish alerts: {trade_count},  and percentage of good bull alerts {(plus_bull_trades / trade_count):.2%}')

All bullish alerts: 198,  and percentage of good bull alerts 52.53%


In [43]:
plus_bear_trades = 0
trade_count = 0
for i, row in hikkake_pattern.iterrows():
    if row['CDLHIKKAKE'] == 'Bearish':
        difference = hikkake_pattern['Close'].iloc[i] - hikkake_pattern['Close'].iloc[i + 1]
        trade_count += 1
        if difference > 0:
            plus_bear_trades += 1
            
print(f'All bearish alerts: {trade_count} and percentage of good bear alerts {(plus_bear_trades / trade_count):.2%}')

All bearish alerts: 236 and percentage of good bear alerts 54.24%


### Gravestone doji

Bearish pattern that suggests a reversal followed by a downtrend in the price action.
- A gravestone doji is a bearish reversal candlestick pattern that is formed when the open, low, and closing prices are all near each other with a long upper shadow.


#### In Ta-lib its marked as "bullish" plus it doesnt look exactly like Gravestone doji from TA literature. But it can stay because these kind of candlestick can be informative combined with other patterns

In [44]:
gravestone_doji = new_df(price, 'CDLGRAVESTONEDOJI')

In [45]:
gravestone_doji.loc[(gravestone_doji['CDLGRAVESTONEDOJI'] == 'Bullish') \
                                      | (gravestone_doji['CDLGRAVESTONEDOJI'] == 'Bearish')].head(15)

,Time,Open,High,Low,Close,CDLGRAVESTONEDOJI,year_week
62,2021-01-11 11:00:00,34372.25,36360.00,34020.00,34198.56,Bullish,2021-02
100,2021-01-17 19:00:00,35728.47,36166.47,35514.99,35633.91,Bullish,2021-03
192,2021-02-02 03:00:00,33517.09,34161.39,33418.00,33550.32,Bullish,2021-05
257,2021-02-12 23:00:00,47272.96,48150.00,47140.26,47287.60,Bullish,2021-06
288,2021-02-18 03:00:00,52117.67,52530.00,51933.24,52032.64,Bullish,2021-07
326,2021-02-24 11:00:00,50090.61,51374.99,49799.29,50360.02,Bullish,2021-08
500,2021-03-25 11:00:00,52270.72,53160.85,52101.00,52163.19,Bullish,2021-12
515,2021-03-27 23:00:00,55765.01,56700.36,55675.00,55817.14,Bullish,2021-12
553,2021-04-03 07:00:00,59221.52,59531.74,59180.00,59298.85,Bullish,2021-13
795,2021-05-13 15:00:00,49777.01,50884.34,49458.68,49627.64,Bullish,2021-19


In [46]:
def bull_gravestone_doji(row):
    
    if row['CDLGRAVESTONEDOJI'] == 'Bullish':
        return row['Open'] * 0.99
    else:
        return np.nan  
    
gravestone_doji['gravestone_doji_bullish_dot'] = gravestone_doji.apply(bull_gravestone_doji, axis=1)

In [47]:
gravestone_doji_plot = gravestone_doji.loc[(gravestone_doji['year_week'] == '2021-24') \
                                          | (gravestone_doji['year_week'] == '2021-25')].copy()

In [48]:
fig = go.Figure()

fig.add_trace(go.Candlestick(x=gravestone_doji_plot['Time'],
                             open=gravestone_doji_plot['Open'],
                             high=gravestone_doji_plot['High'],
                             low=gravestone_doji_plot['Low'],
                             close=gravestone_doji_plot['Close'], name="Price", opacity=0.5))


fig.add_trace(go.Scatter(x=gravestone_doji_plot['Time'], 
                         y=gravestone_doji_plot['gravestone_doji_bullish_dot'],
                         opacity=1, mode='markers', marker=dict(color='#16FF32', symbol='5', size=7, line=dict(color='MediumPurple',width=2)),
                         name="Bullish"))


fig.update_layout(xaxis_rangeslider_visible=False, title = 'Gravestone doji')
iplot(fig, show_link=False)

![pattern](https://lh3.googleusercontent.com/9RSWZs1wsRreezKZhOkNbYt_oWEV-zrpw1SnGgU37rV-dVu7KqXOVxY-5gWXbPpv2jcs8DMsJqZOwlg3emHE0AhtyBzP8wUXxmyhVO-C6_FGmhXesOfzb5rliM0PDRBvNaJzzgIAM6jd7PT8dBXzTQU)

As you can see, patterns do not always predict the moments of a reversal, so you should not rely entirely on them. Now lets calculate how many times our patterns appear per week.

In [49]:
def stat_patterns_coin(selected_patterns, df_price):
    
    """
    how often do patterns appear during the week
    """
    
    coin_stat = df_price[selected_patterns].copy()
    coin_stat['year_week'] = pd.to_datetime(df_price['Time']).dt.strftime('%Y-%U')
    coin_stat.replace('0', np.nan, inplace=True)
    
    patterns_per_week = coin_stat.groupby(['year_week'], as_index=False).count()
    
    df_stat = {}

    for pattern in selected_patterns:
        df_stat[pattern] = round(patterns_per_week[pattern].mean(), 2)
    
    return pd.Series(df_stat, name='weekly_alerts').sort_values(ascending=False)

Some patterns are rare, while two of them can appear almost every day. 

In [50]:
stat_patterns_coin(btc50['name'].tolist(), price)

CDLHIKKAKE             6.68
CDLHIGHWAVE            6.60
CDLHARAMI              1.67
CDLMATCHINGLOW         0.79
CDLGRAVESTONEDOJI      0.57
CDLDOJISTAR            0.37
CDLHANGINGMAN          0.35
CDLXSIDEGAP3METHODS    0.24
CDLSHOOTINGSTAR        0.16
CDL3LINESTRIKE         0.12
CDLEVENINGDOJISTAR     0.01
Name: weekly_alerts, dtype: float64

## Altcoins Summary<a id='4-bullet'></a>

### Ethereum (ETH)

In [51]:
eth50 = pd.read_csv('eth50.csv')
eth50

,name,prc_good_trades,num_trades
0,CDLINVERTEDHAMMER,64.00,9.0
1,CDLXSIDEGAP3METHODS,63.50,17.0
2,CDL3LINESTRIKE,62.50,10.0
3,CDLDOJISTAR,58.50,25.0
4,CDLMORNINGDOJISTAR,58.50,3.0
5,CDLGAPSIDESIDEWHITE,58.50,6.0
6,CDLHAMMER,55.50,74.0
7,CDLRICKSHAWMAN,53.75,416.0
8,CDLMATCHINGLOW,53.50,48.0
9,CDLDOJI,53.25,516.0


In [52]:
df_eth = pd.read_csv('eth_patterns_4h.csv')
stat_patterns_coin(eth50['name'].tolist(), df_eth)

CDLHIKKAKE             6.53
CDLDOJI                6.38
CDLLONGLEGGEDDOJI      6.37
CDLRICKSHAWMAN         5.15
CDLHAMMER              0.91
CDLMATCHINGLOW         0.59
CDLHANGINGMAN          0.56
CDLDOJISTAR            0.32
CDLXSIDEGAP3METHODS    0.21
CDL3LINESTRIKE         0.12
CDLINVERTEDHAMMER      0.11
CDLGAPSIDESIDEWHITE    0.07
CDLMORNINGDOJISTAR     0.04
Name: weekly_alerts, dtype: float64

### Solana (SOL)

In [53]:
sol50 = pd.read_csv('sol50.csv')
sol50

,name,prc_good_trades,num_trades
0,CDLDARKCLOUDCOVER,100.00,1.0
1,CDLEVENINGDOJISTAR,100.00,1.0
2,CDLEVENINGSTAR,73.25,13.0
3,CDLSTALLEDPATTERN,58.00,13.0
4,CDL3LINESTRIKE,55.50,14.0
5,CDLGRAVESTONEDOJI,53.25,39.0
6,CDLRICKSHAWMAN,52.50,363.0
7,CDLHIKKAKE,51.75,431.0
8,CDLDOJI,51.00,462.0
9,CDLLONGLEGGEDDOJI,50.75,461.0


In [54]:
df_sol = pd.read_csv('sol_patterns_4h.csv')
stat_patterns_coin(sol50['name'].tolist(), df_sol)

CDLSPINNINGTOP        9.43
CDLHIKKAKE            6.62
CDLDOJI               5.72
CDLLONGLEGGEDDOJI     5.70
CDLHIGHWAVE           5.46
CDLRICKSHAWMAN        4.49
CDLGRAVESTONEDOJI     0.48
CDL3LINESTRIKE        0.17
CDLEVENINGSTAR        0.16
CDLSTALLEDPATTERN     0.16
CDLDARKCLOUDCOVER     0.01
CDLEVENINGDOJISTAR    0.01
Name: weekly_alerts, dtype: float64

### Dogecoin (DOGE)

In [55]:
doge50 = pd.read_csv('doge50.csv')
doge50

,name,prc_good_trades,num_trades
0,CDLGAPSIDESIDEWHITE,87.50,2.0
1,CDLEVENINGSTAR,62.50,10.0
2,CDLSHOOTINGSTAR,61.75,19.0
3,CDLINVERTEDHAMMER,60.25,12.0
4,CDL3LINESTRIKE,59.00,8.0
5,CDLADVANCEBLOCK,56.25,24.0
6,CDLHANGINGMAN,55.00,25.0
7,CDLMATCHINGLOW,52.50,62.0
8,CDLHIKKAKE,51.75,502.5
9,CDLGRAVESTONEDOJI,51.50,66.0


In [56]:
df_doge = pd.read_csv('doge_patterns_4h.csv')
stat_patterns_coin(doge50['name'].tolist(), df_doge)

CDLHIKKAKE             7.40
CDLHAMMER              0.94
CDLGRAVESTONEDOJI      0.81
CDLMATCHINGLOW         0.77
CDLHANGINGMAN          0.31
CDLADVANCEBLOCK        0.30
CDLSHOOTINGSTAR        0.23
CDLXSIDEGAP3METHODS    0.17
CDLINVERTEDHAMMER      0.15
CDLEVENINGSTAR         0.12
CDL3LINESTRIKE         0.10
CDLGAPSIDESIDEWHITE    0.02
Name: weekly_alerts, dtype: float64

### Near (NEAR)

In [57]:
near50 = pd.read_csv('near50.csv')
near50

,name,prc_good_trades,num_trades
0,CDL3WHITESOLDIERS,100.00,1.0
1,CDLINNECK,100.00,1.0
2,CDLIDENTICAL3CROWS,75.00,2.0
3,CDLTASUKIGAP,75.00,1.0
4,CDL3LINESTRIKE,60.50,17.0
5,CDLGAPSIDESIDEWHITE,58.50,3.0
6,CDLMORNINGSTAR,56.00,16.0
7,CDLHIKKAKEMOD,55.00,10.0
8,CDLSHOOTINGSTAR,53.75,20.0
9,CDLHARAMI,53.00,165.0


In [58]:
df_near = pd.read_csv('near_patterns_4h.csv')
stat_patterns_coin(near50['name'].tolist(), df_near)

CDLHIKKAKE             6.84
CDLHIGHWAVE            6.02
CDLHARAMI              2.04
CDLHAMMER              0.95
CDLMATCHINGLOW         0.70
CDLDOJISTAR            0.59
CDLADVANCEBLOCK        0.53
CDLHANGINGMAN          0.38
CDLSHOOTINGSTAR        0.25
CDLINVERTEDHAMMER      0.23
CDL3LINESTRIKE         0.21
CDLMORNINGSTAR         0.20
CDLSEPARATINGLINES     0.20
CDLHIKKAKEMOD          0.16
CDLGAPSIDESIDEWHITE    0.04
CDLIDENTICAL3CROWS     0.02
CDLINNECK              0.01
CDLTASUKIGAP           0.01
CDL3WHITESOLDIERS      0.01
Name: weekly_alerts, dtype: float64

### Harmony (ONE)

In [59]:
one50 = pd.read_csv('one50.csv')
one50

,name,prc_good_trades,num_trades
0,CDL3BLACKCROWS,100.00,1.0
1,CDLIDENTICAL3CROWS,75.00,2.0
2,CDLMORNINGDOJISTAR,67.00,3.0
3,CDL3LINESTRIKE,65.00,10.0
4,CDLDOJISTAR,60.50,42.0
5,CDLHARAMICROSS,59.75,42.0
6,CDLHOMINGPIGEON,58.25,3.0
7,CDL3INSIDE,55.25,23.0
8,CDLHIKKAKE,53.00,440.0
9,CDLMARUBOZU,51.75,112.0


In [60]:
df_one = pd.read_csv('one_patterns_4h.csv')
stat_patterns_coin(one50['name'].tolist(), df_one)

CDLSPINNINGTOP        10.04
CDLHIKKAKE             6.81
CDLCLOSINGMARUBOZU     4.09
CDLMARUBOZU            1.38
CDLMATCHINGLOW         0.78
CDLDOJISTAR            0.52
CDLHARAMICROSS         0.52
CDLHANGINGMAN          0.36
CDL3INSIDE             0.28
CDL3LINESTRIKE         0.12
CDLMORNINGDOJISTAR     0.04
CDLHOMINGPIGEON        0.04
CDLIDENTICAL3CROWS     0.02
CDL3BLACKCROWS         0.01
Name: weekly_alerts, dtype: float64

### Binance coin (BNB)

In [61]:
bnb50 = pd.read_csv('bnb50.csv')
bnb50

,name,prc_good_trades,num_trades
0,CDLMORNINGDOJISTAR,87.50,4.0
1,CDLEVENINGDOJISTAR,83.50,3.0
2,CDL3WHITESOLDIERS,75.00,4.0
3,CDLHOMINGPIGEON,75.00,1.0
4,CDLMORNINGSTAR,72.50,10.0
5,CDLHARAMICROSS,63.00,31.0
6,CDL3INSIDE,61.75,19.0
7,CDLEVENINGSTAR,61.50,13.0
8,CDL3LINESTRIKE,56.25,8.0
9,CDLSHOOTINGSTAR,56.00,17.0


In [62]:
df_bnb = pd.read_csv('bnb_patterns_4h.csv')
stat_patterns_coin(bnb50['name'].tolist(), df_bnb)

CDLLONGLINE           7.29
CDLHIKKAKE            6.33
CDLCLOSINGMARUBOZU    4.66
CDLHARAMI             1.60
CDLMARUBOZU           1.41
CDLHAMMER             1.02
CDLHANGINGMAN         0.71
CDLHARAMICROSS        0.38
CDL3INSIDE            0.23
CDLSHOOTINGSTAR       0.21
CDLEVENINGSTAR        0.16
CDLSTALLEDPATTERN     0.16
CDLMORNINGSTAR        0.12
CDL3LINESTRIKE        0.10
CDL3WHITESOLDIERS     0.05
CDLMORNINGDOJISTAR    0.05
CDLEVENINGDOJISTAR    0.04
CDLHOMINGPIGEON       0.01
Name: weekly_alerts, dtype: float64

### Ripple (XRP)

In [63]:
xrp50 = pd.read_csv('xrp50.csv')
xrp50

,name,prc_good_trades,num_trades
0,CDLEVENINGSTAR,71.00,7.0
1,CDLSEPARATINGLINES,70.75,11.0
2,CDLSTALLEDPATTERN,62.50,4.0
3,CDLSHOOTINGSTAR,59.25,22.0
4,CDLGRAVESTONEDOJI,56.00,48.0
5,CDLXSIDEGAP3METHODS,55.25,14.0
6,CDLMATCHINGLOW,54.25,62.0
7,CDLDOJI,53.75,553.5
8,CDLLONGLEGGEDDOJI,53.75,552.5
9,CDLRICKSHAWMAN,53.00,409.5


In [64]:
df_xrp = pd.read_csv('xrp_patterns_4h.csv')
stat_patterns_coin(xrp50['name'].tolist(), df_xrp)

CDLHIKKAKE             7.38
CDLDOJI                6.76
CDLLONGLEGGEDDOJI      6.74
CDLRICKSHAWMAN         5.00
CDLMATCHINGLOW         0.76
CDLDRAGONFLYDOJI       0.73
CDLTAKURI              0.73
CDLGRAVESTONEDOJI      0.59
CDLSHOOTINGSTAR        0.27
CDLHARAMICROSS         0.27
CDLXSIDEGAP3METHODS    0.17
CDLHIKKAKEMOD          0.15
CDLSEPARATINGLINES     0.13
CDLEVENINGSTAR         0.09
CDLSTALLEDPATTERN      0.05
Name: weekly_alerts, dtype: float64

### Cardano (ADA)

In [65]:
ada50 = pd.read_csv('ada50.csv')
ada50

,name,prc_good_trades,num_trades
0,CDLTASUKIGAP,100.00,1.0
1,CDLSTICKSANDWICH,75.00,1.0
2,CDLGAPSIDESIDEWHITE,70.75,6.0
3,CDLHANGINGMAN,64.75,22.0
4,CDLINVERTEDHAMMER,61.75,15.0
5,CDLXSIDEGAP3METHODS,57.25,28.0
6,CDLDOJISTAR,52.50,24.0
7,CDLMARUBOZU,52.25,95.0
8,CDLSHOOTINGSTAR,52.00,12.0
9,CDLMATCHINGLOW,51.00,50.0


In [66]:
df_ada = pd.read_csv('ada_patterns_4h.csv')
stat_patterns_coin(ada50['name'].tolist(), df_ada)

CDLMARUBOZU            1.16
CDLMATCHINGLOW         0.61
CDLXSIDEGAP3METHODS    0.34
CDLDOJISTAR            0.30
CDLHANGINGMAN          0.27
CDLINVERTEDHAMMER      0.18
CDLSHOOTINGSTAR        0.15
CDLSTALLEDPATTERN      0.13
CDLGAPSIDESIDEWHITE    0.07
CDLTASUKIGAP           0.01
CDLSTICKSANDWICH       0.01
Name: weekly_alerts, dtype: float64

### Cosmos (Atom)

In [67]:
atom50 = pd.read_csv('atom50.csv')
atom50

,name,prc_good_trades,num_trades
0,CDLPIERCING,75.00,1.0
1,CDLEVENINGDOJISTAR,65.00,5.0
2,CDL3LINESTRIKE,61.50,9.0
3,CDL3WHITESOLDIERS,58.50,3.0
4,CDLSTALLEDPATTERN,57.50,10.0
5,CDLEVENINGSTAR,56.25,16.0
6,CDLINVERTEDHAMMER,56.00,13.0
7,CDLHARAMI,53.75,147.0
8,CDLMARUBOZU,52.50,126.0
9,CDLHANGINGMAN,52.00,48.0


In [68]:
df_atom = pd.read_csv('atom_patterns_4h.csv')
stat_patterns_coin(atom50['name'].tolist(), df_atom)

CDLSHORTLINE          6.27
CDLCLOSINGMARUBOZU    4.80
CDLRICKSHAWMAN        4.55
CDLHARAMI             1.79
CDLMARUBOZU           1.54
CDLHANGINGMAN         0.59
CDLEVENINGSTAR        0.20
CDLINVERTEDHAMMER     0.16
CDLSTALLEDPATTERN     0.12
CDL3LINESTRIKE        0.11
CDLEVENINGDOJISTAR    0.06
CDL3WHITESOLDIERS     0.04
CDLPIERCING           0.01
Name: weekly_alerts, dtype: float64

### Fantom (FTT)

In [69]:
ftt50 = pd.read_csv('ftt50.csv')
ftt50

,name,prc_good_trades,num_trades
0,CDLIDENTICAL3CROWS,100.00,2.0
1,CDLRISEFALL3METHODS,100.00,1.0
2,CDLTHRUSTING,100.00,1.0
3,CDLLADDERBOTTOM,100.00,1.0
4,CDLHIKKAKEMOD,62.50,2.0
5,CDLMORNINGSTAR,59.75,13.0
6,CDLSHOOTINGSTAR,59.75,29.0
7,CDL3LINESTRIKE,55.25,14.0
8,CDLEVENINGDOJISTAR,54.25,6.0
9,CDLCLOSINGMARUBOZU,54.25,449.0


In [70]:
df_ftt = pd.read_csv('ftt_patterns_4h.csv')
stat_patterns_coin(ftt50['name'].tolist(), df_ftt)

CDLLONGLINE            7.73
CDLDOJI                5.66
CDLHIKKAKE             5.61
CDLCLOSINGMARUBOZU     5.48
CDLRICKSHAWMAN         4.49
CDLMARUBOZU            1.91
CDL3OUTSIDE            1.77
CDLMATCHINGLOW         0.56
CDLDOJISTAR            0.54
CDLSHOOTINGSTAR        0.35
CDLEVENINGSTAR         0.22
CDL3LINESTRIKE         0.17
CDLMORNINGSTAR         0.16
CDLMORNINGDOJISTAR     0.09
CDLEVENINGDOJISTAR     0.07
CDLHIKKAKEMOD          0.02
CDLIDENTICAL3CROWS     0.02
CDLRISEFALL3METHODS    0.01
CDLLADDERBOTTOM        0.01
CDLTHRUSTING           0.01
Name: weekly_alerts, dtype: float64

### Litecoin (LTC)

In [71]:
ltc50 = pd.read_csv('ltc50.csv')
ltc50

,name,prc_good_trades,num_trades
0,CDLDOJISTAR,62.50,26.0
1,CDLHANGINGMAN,56.00,46.0
2,CDLMORNINGSTAR,55.00,10.0
3,CDLHARAMICROSS,54.25,29.0
4,CDLHAMMER,54.00,77.0
5,CDLEVENINGSTAR,53.75,19.0
6,CDL3INSIDE,53.00,24.0
7,CDLHIKKAKE,52.50,368.0
8,CDL3LINESTRIKE,52.50,10.0
9,CDLDOJI,51.75,511.0


In [72]:
df_ltc = pd.read_csv('ltc_patterns_4h.csv')
stat_patterns_coin(ltc50['name'].tolist(), df_ltc)

CDLDOJI              6.24
CDLLONGLEGGEDDOJI    6.23
CDLHIKKAKE           5.51
CDLRICKSHAWMAN       4.90
CDLHARAMI            1.41
CDLHAMMER            0.94
CDLHANGINGMAN        0.56
CDLHARAMICROSS       0.35
CDLDOJISTAR          0.32
CDL3INSIDE           0.29
CDLEVENINGSTAR       0.23
CDLMORNINGSTAR       0.12
CDL3LINESTRIKE       0.12
Name: weekly_alerts, dtype: float64

### Avalanche (AVAX)

In [73]:
avax50 = pd.read_csv('avax50.csv')
avax50

,name,prc_good_trades,num_trades
0,CDLGAPSIDESIDEWHITE,100.00,3.0
1,CDLMORNINGDOJISTAR,66.75,6.0
2,CDLMORNINGSTAR,61.75,15.0
3,CDLDOJISTAR,59.50,37.0
4,CDLHARAMICROSS,59.00,30.0
5,CDLSEPARATINGLINES,58.75,14.0
6,CDLTAKURI,57.75,54.0
7,CDLHANGINGMAN,57.50,43.0
8,CDLDRAGONFLYDOJI,56.50,56.0
9,CDLSHOOTINGSTAR,55.00,24.0


In [74]:
df_avax = pd.read_csv('avax_patterns_4h.csv')
stat_patterns_coin(avax50['name'].tolist(), df_avax)

CDLSPINNINGTOP         9.11
CDLHIKKAKE             6.21
CDLHIGHWAVE            5.41
CDLHARAMI              1.70
CDLDRAGONFLYDOJI       0.68
CDLTAKURI              0.66
CDLHANGINGMAN          0.52
CDLDOJISTAR            0.45
CDLHARAMICROSS         0.37
CDLXSIDEGAP3METHODS    0.33
CDLSHOOTINGSTAR        0.29
CDLMORNINGSTAR         0.18
CDLSEPARATINGLINES     0.17
CDLINVERTEDHAMMER      0.16
CDLSTALLEDPATTERN      0.12
CDLMORNINGDOJISTAR     0.07
CDLGAPSIDESIDEWHITE    0.04
Name: weekly_alerts, dtype: float64

### Polkadot (DOT)

In [75]:
dot50 = pd.read_csv('dot50.csv')
dot50

,name,prc_good_trades,num_trades
0,CDLIDENTICAL3CROWS,100.00,2.0
1,CDLHANGINGMAN,64.00,37.0
2,CDLSEPARATINGLINES,61.75,13.0
3,CDLMORNINGSTAR,55.00,10.0
4,CDLHIKKAKE,53.50,417.0
5,CDLRICKSHAWMAN,51.50,364.0
6,CDLBELTHOLD,50.50,432.0
7,CDLHIGHWAVE,50.25,455.0


In [76]:
df_dot = pd.read_csv('dot_patterns_4h.csv')
stat_patterns_coin(dot50['name'].tolist(), df_dot)

CDLHIKKAKE            6.52
CDLHIGHWAVE           5.55
CDLBELTHOLD           5.27
CDLRICKSHAWMAN        4.44
CDLHANGINGMAN         0.45
CDLSEPARATINGLINES    0.16
CDLMORNINGSTAR        0.12
CDLIDENTICAL3CROWS    0.02
Name: weekly_alerts, dtype: float64

### Polygon (MATIC)

In [77]:
matic50 = pd.read_csv('matic50.csv')
matic50

,name,prc_good_trades,num_trades
0,CDLEVENINGDOJISTAR,100.00,1.0
1,CDL3WHITESOLDIERS,100.00,1.0
2,CDLHOMINGPIGEON,100.00,1.0
3,CDLRISEFALL3METHODS,87.50,2.0
4,CDL3LINESTRIKE,65.50,8.0
5,CDL3INSIDE,64.00,23.5
6,CDLINVERTEDHAMMER,62.50,14.0
7,CDLHARAMICROSS,58.00,41.0
8,CDLHANGINGMAN,56.25,35.0
9,CDLHIKKAKEMOD,54.00,6.0


In [78]:
df_matic = pd.read_csv('matic_patterns_4h.csv')
stat_patterns_coin(matic50['name'].tolist(), df_matic)

CDLHARAMI              1.85
CDLMATCHINGLOW         0.65
CDLHARAMICROSS         0.50
CDLHANGINGMAN          0.43
CDL3INSIDE             0.29
CDLSHOOTINGSTAR        0.22
CDLXSIDEGAP3METHODS    0.22
CDLINVERTEDHAMMER      0.17
CDLHIKKAKEMOD          0.12
CDL3LINESTRIKE         0.10
CDLRISEFALL3METHODS    0.02
CDLEVENINGDOJISTAR     0.01
CDL3WHITESOLDIERS      0.01
CDLHOMINGPIGEON        0.01
Name: weekly_alerts, dtype: float64

## Results <a id='5-bullet'></a>

The most popular patterns are found on almost all coins: Hikkake and Hanging Man, while 10 of the selected patterns perform well on only one coin. Most patterns will be searched for FTT, while BTC is in the top three coins with the least number of patterns.
&nbsp;<br>
&nbsp;<br>

In [79]:
pdList = [one50, sol50, near50, doge50, btc50, 
          eth50, bnb50, xrp50, ada50, dot50, 
          atom50, ftt50, ltc50, avax50, matic50]

new_df_all_50 = pd.concat(pdList)

new_df_all_50['name'].value_counts().head()

CDLHIKKAKE         12
CDLHANGINGMAN      12
CDL3LINESTRIKE     11
CDLSHOOTINGSTAR     9
CDLMATCHINGLOW      9
Name: name, dtype: int64

In [80]:
print(f"Total number of patterns selected: {len(new_df_all_50['name'].value_counts().index)}")

Total number of patterns selected: 47


In [81]:
fig = go.Figure()

fig.add_trace(go.Bar(x=new_df_all_50['name'].value_counts().index, 
                         y=new_df_all_50['name'].value_counts().values))


fig.update_layout(title = 'Selected patterns')
fig.update_xaxes(tickangle=45, tickfont=dict(family='Rockwell', size=14))
iplot(fig, show_link=False)

![selected](https://lh4.googleusercontent.com/qJdvSy3MFe3P9x7Q2Wm-ByXmLJXGd2W3sHlenpjmPHkQv0hbaTxScHwlLbMyH9Q_F5kEayMfvVVhqdpJpAONJe5QoMZCViKI3ggmbokSMgYUCCa-d2euV2ugyUEW3IO3TjRzmP9WgqvXKv1gN9HScZQ)


![selected](https://lh5.googleusercontent.com/XhxL-hMQNdwXLnprC7PEczaHbgd3NSxSAYiszaZ5j4jeBPybFTFMEFG8bjB0dLwvx4XNiODGfrGuoTocR_2B35sCvPPb8IY-golYOuBOgiV1By9weH2xLVn2EcwAA3CsielvmFlM34WUiSqyzKgeO7A)